### Setup

In [31]:
import os
import dotenv
import json
from pydantic import BaseModel, Field, validator
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.output_parsers import PydanticOutputParser
from jobgpt.resume_analyzer.resume_reader import ResumeReader
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from langchain.chains import SequentialChain
import langchain
from jobgpt.utils.llm import count_tokens
from typing import List
import difflib
dotenv.load_dotenv(".env", override=True)
from langchain.schema import AIMessage, HumanMessage, SystemMessage

In [38]:
chat = ChatOpenAI(temperature=0)

In [39]:
template = (
    "You are a helpful assistant that translates {input_language} to {output_language}."
)
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt, human_message_prompt]
)

# get a chat completion from the formatted messages
chat(
    chat_prompt.format_prompt(
        input_language="English", output_language="French", text="I love programming."
    ).to_messages()
)


AIMessage(content="J'adore la programmation.", additional_kwargs={}, example=False)

In [2]:
# langchain.debug = True

In [3]:
class WorkExperience(BaseModel):
    title: str = Field(description="The title of the position")
    company: str = Field(description="The company of the position")
    bullet_points: List[str] = Field(description="The list of text that describes the experience")

class Education(BaseModel):
    school: str = Field(description="The school of the education")
    degree: str = Field(description="The degree of the education")
    major: str = Field(description="The major of the education")
    bullet_points: List[str] = Field(description="The list of text that describes the education")

class PersonalProject(BaseModel):
    title: str = Field(description="The title of the project")
    bullet_points: List[str] = Field(description="The list of text that describes the project and the canditate's contribution")

class Summary(BaseModel):
    text: str = Field(description="The summary or highlight of the resume which usually appears at the top or buttom of the resume")

class Skills(BaseModel):
    text: str = Field(description="The list of skills that the candidate has")
# class ResumeAnalysis(BaseModel):
#     evaluation: str = Field(description="The overall evaluation of the resume")
#     suggestions: List[str] = Field(description="Suggestions for the resume, format into list")
#     revised_resume: List[ResumeSection] = Field(description="The revised resume")

In [4]:
# initialize the models
openai = ChatOpenAI(
    model_name="gpt-4",
    openai_api_key=os.environ["OPENAI_API_KEY"],
    temperature=0.7,
    verbose=True,
)

In [5]:
# read text from file
with open("../data/job_descriptions/jd_1.txt", "r") as f:
    job_description = f.read()

In [6]:
# template_read = """
# You are given a text of resume, extract the key fields from the resume and output them in a JSON format.
# Make sure to EXCLUDE the fileds such as personal information, contact information, and times.

# Resume Text: {resume_text}
# Answer:
# """
# prompt_read = PromptTemplate(input_variables=["resume_text"], template=template_read)
# chain_read = LLMChain(llm=openai, prompt=prompt_read)
# resume = count_tokens(chain_read, {"resume_text": resume_text})
# print(resume["token_count"])
# resume = json.loads(resume["result"])

In [7]:
# from langchain.output_parsers import ResponseSchema
# from langchain.output_parsers import StructuredOutputParser
# evaluation_schema = ResponseSchema(name="evaluation", description="The overall evaluation of the resume")
# suggestions_schema = ResponseSchema(name="suggestions", description="Suggestions for the resume, format into list")
# revised_resume_schema = ResponseSchema(name="revised_resume", description="The revised resume")
# response_schemas = [evaluation_schema, 
#                     suggestions_schema,
#                     revised_resume_schema]
# output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
# format_instructions = output_parser.get_format_instructions()
# print(format_instructions)


In [8]:
parser = PydanticOutputParser(pydantic_object=ResumeAnalysis)

In [9]:
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"evaluation": {"title": "Evaluation", "description": "The overall evaluation of the resume", "type": "string"}, "suggestions": {"title": "Suggestions", "description": "Suggestions for the resume, format into list", "type": "array", "items": {"type": "string"}}, "revised_resume": {"title": "Revised Resume", "description": "The revised resume", "type": "array", "items": {"$ref": "#/definitions/ResumeSection"}}}, "required": ["evaluation", "suggestions", "revised_resume"], "definitions": {"ResumeSection": {"title": "ResumeSection

### Strategy 1

In [7]:
resume_text = ResumeReader().read("../data/resumes/resume_4.pdf")

In [6]:
system_template = """
You are an experienced career consultalt who helps clients to improve their resumes.
When you are asked to provide evaluation or suggestion, make sure your are critical and specific.
Focus on the use of professional language and the relevancy to the job description.
REMEMBER DO NOT make things up or create fake experiences. 

You response should be in JSON format with following keys: "evaluation", "suggestions", "revised_resume".
"""
user_teamplate = """
You are an experienced career consultalt helping clients with their resumes.
Let's think step by step and section by secion.
First, give an CRITICAL evaluation of the resume focusing on use of professional language and the relevancy to the job description.
Second, provide suggestions on how the client can improve the section. Mention the exact wording used and how the client can reword it. 
Try to list all the problems and give specific suggestions.
Last, give a revision of the work experience section besed on your suggestions.

resume: {resume_text}
Job Description: {job_description}
"""
system_prompt = SystemMessagePromptTemplate.from_template(system_template.strip())
user_prompt = HumanMessagePromptTemplate.from_template(user_teamplate.strip())
resume_analyzer_prompt_1 = ChatPromptTemplate(input_variables=["resume_text", "job_description"], messages=[system_prompt, user_prompt])
chain_analyze_1 = LLMChain(llm=openai, prompt=resume_analyzer_prompt_1)

In [7]:
output = count_tokens(chain_analyze_1, {"resume_text": resume_text, "job_description": job_description})
print(output["token_count"])

2139


In [10]:
print("===================output======================")
print(output['result'])
output_json = json.loads(output['result'])
print("===================resume======================")
print(output_json["revised_resume"])

===================output======================
{
  "evaluation": "The resume demonstrates a good foundation in terms of work experience and education. However, there are areas where the language can be improved to better align with the job description. The resume could benefit from more specific examples of credit risk and financial analysis experience, as well as highlighting any industry expertise. Additionally, the use of professional language can be enhanced in some areas.",
  "suggestions": [
    "In the Credit Analyst role, consider rewording 'Manage approval pipeline' to 'Managed credit approval pipeline' to emphasize credit risk experience.",
    "In the Head Customer Service Representative role, replace 'improving customer experience' with 'exceptional customer service' to better align with the job description's language.",
    "In the Teaching Assistant role, consider adding specific examples of financial analysis or credit risk-related tasks, if applicable.",
    "Under the

In [12]:
import textwrap
print(textwrap.fill(output_json['evaluation'], 50))

The resume demonstrates a good foundation in terms
of work experience and education. However, there
are areas where the language can be improved to
better align with the job description. The resume
could benefit from more specific examples of
credit risk and financial analysis experience, as
well as highlighting any industry expertise.
Additionally, the use of professional language can
be enhanced in some areas.


### Strategy 2

In [10]:
resume_work_experience = """
Credit AnalystVine Group, Toronto ON/BNB Financial Group (Merger)                                        Oct 2020 -Present•Manage approval pipeline through daily review and regular follow-up to collect outstanding documentation to ensure mortgages are funding on time•Review documentation for completeness in highest credit quality standards, with compliance to lender policies and procedures•Provide smooth customer service experience in integration with clients and agent team by managing and optimizing interactions•Establish and maintain positive business relationship with external business partners Head Customer Service RepresentativeCIBCMasonville Banking Centre,London,ON                                                    Oct 2018 - Oct 2020•Consistently provided improving customer experience while ensuring error-free transactions in fast pace environment•Assisted CSR team with reconciling and balancing and efficiently correctingerrors•Supported CSR team in balancing system update, proactively created chit-sheet to avoid balancing errors for team•Successfully achieved quality introduction results at 198% over target referrals on SmartReport in 2019•Proactively identified opportunities to introduce client to FSR team to develop financial plansTeaching Assistant     Department of Economics and Finance, UniversityofGuelph                           Sept 2017 - April 2018•Assisted with grading, conducted tutorial sessions and scheduled office hours with students to address course relatedquestions•Worked with minimal supervision while setting priorities and adapting to multiple demands efficiently•Answered students’ questions in a timely manner
"""

In [27]:
system_template = """
You are an experienced career consultalt who helps clients to improve their resumes.
When you are asked to provide evaluation or suggestion, make sure your are critical and specific.
Focus on the use of professional language and the relevancy to the job description.
REMEMBER DO NOT make things up or create fake experiences. 

You response should be in JSON format with following keys: "evaluation", "suggestions", "revised_resume".
"""
user_teamplate = """
You are a experienced career consultalt helping clients with the {section} section of their resumes given a job description that the client is applying for.
Let's think step by step and experience by experience.
First, give an CRITICAL evaluation of the {section} focusing on use of professional language and the relevancy to the job description.
Second, provide suggestions on how the client can improve the section. Mention the exact wording used and how the client can reword it. 
Try to find all the problems and give specific suggestions.
Last, give a revision of the {section} section besed on your suggestions.

{section}: {section_text}
Job Description: {job_description}
"""
system_prompt = SystemMessagePromptTemplate.from_template(system_template.strip())
user_prompt = HumanMessagePromptTemplate.from_template(user_teamplate.strip())
resume_analyzer_prompt_2 = ChatPromptTemplate(input_variables=["section", "section_text", "job_description"], messages=[system_prompt, user_prompt])
chain_analyze_2 = LLMChain(llm=openai, prompt=resume_analyzer_prompt_2)

In [11]:
system_template = """
You are an experienced career consultalt who helps clients to improve their resumes.
When you are asked to provide evaluation or suggestion, make sure your are critical and specific.
Focus on the use of professional language and the relevancy to the job description.
REMEMBER DO NOT make things up or create fake experiences. 
{json_format}
"""
user_teamplate = """
You are a experienced career consultalt helping clients with the {section} section of their resumes given a job description that the client is applying for.
Let's think step by step and experience by experience.
First, give an CRITICAL evaluation of the {section} focusing on use of professional language and the relevancy to the job description.
Second, provide suggestions on how the client can improve the section. Mention the exact wording used and how the client can reword it. 
Try to find all the problems and give specific suggestions.
Last, give a revision of the {section} section besed on your suggestions.

{section}: {section_text}
Job Description: {job_description}
JSON response:
"""
system_prompt = SystemMessagePromptTemplate.from_template(system_template.strip())
user_prompt = HumanMessagePromptTemplate.from_template(user_teamplate.strip())
resume_analyzer_prompt_2 = ChatPromptTemplate(input_variables=["json_format", "section", "section_text", "job_description"], messages=[system_prompt, user_prompt])
chain_analyze_2 = LLMChain(llm=openai, prompt=resume_analyzer_prompt_2)

In [12]:
# get promt text from prompt template
messages = resume_analyzer_prompt_2.format_prompt(
    json_format=parser.get_format_instructions(),
    section="work experience", 
    section_text=resume_work_experience, 
    job_description=job_description).to_messages()

In [13]:
output = count_tokens(chain_analyze_2, 
                      {"json_format": parser.get_format_instructions(),
                       "section": "work experience", 
                       "section_text": resume_work_experience, 
                       "job_description": job_description})
print(output["token_count"])

1879


In [16]:
print("===================output======================")
print(output['result'])
output_json = json.loads(output['result'])
print("===================resume======================")
print(output_json["revised_resume"])

===================output======================
{
  "evaluation": "The work experience section lacks proper formatting and some relevant experiences are not highlighted effectively. Professional language can be improved.",
  "suggestions": [
    "Use consistent formatting for dates, company names, and locations.",
    "Reword 'Manage approval pipeline' to 'Manage mortgage approval pipeline'.",
    "Replace 'highest credit quality standards' with 'strict credit quality standards'.",
    "Reword 'Provide smooth customer service experience' to 'Ensure seamless customer service experience'.",
    "Add experience related to credit risk analysis and financial statement assessment.",
    "Mention any experience with S&P, Capital IQ, or Moody's, if applicable.",
    "Highlight experience in coaching and mentoring junior team members, if any."
  ],
  "revised_resume": [
    {
      "title": "Credit Analyst",
      "company": "Vine Group, Toronto, ON / BNB Financial Group (Merger)",
      "text"

In [42]:
output_json["revised_resume"]

[{'title': 'Credit Analyst',
  'company': 'Vine Group, Toronto ON / BNB Financial Group (Merger)',
  'text': ['Managed credit risk by reviewing mortgage documentation for compliance with lender policies and procedures, ensuring the highest credit quality standards.',
   "Analyzed financial statements to assess borrower's creditworthiness.",
   'Collaborated with clients and agent team to provide smooth customer service experience and optimized interactions.',
   'Established and maintained positive business relationships with external business partners.']},
 {'title': 'Head Customer Service Representative',
  'company': 'CIBC Masonville Banking Centre, London, ON',
  'text': ['Consistently improved customer experience while ensuring error-free transactions in a fast-paced environment.',
   'Assisted the CSR team with reconciling and balancing, efficiently correcting errors.',
   'Supported the CSR team in system updates and proactively created resources to avoid balancing errors.',
   

In [17]:
text1 = "Manage approval pipeline through daily review and regular follow-up to collect outstanding documentation to ensure mortgages are funding on time"
text2 = "Managed credit risk by reviewing mortgage documentation for compliance with lender policies and procedures, ensuring the highest credit quality standards."

In [23]:
d = difflib.Differ()
diff = d.compare(text1.split(), text2.split())

In [24]:
for line in diff:
    print(line)

- Manage
+ Managed
?       +

+ credit
+ risk
+ by
- approval
- pipeline
- through
- daily
- review
+ reviewing
?       +++

+ mortgage
+ documentation
+ for
+ compliance
+ with
+ lender
+ policies
  and
+ procedures,
+ ensuring
+ the
+ highest
+ credit
+ quality
+ standards.
- regular
- follow-up
- to
- collect
- outstanding
- documentation
- to
- ensure
- mortgages
- are
- funding
- on
- time


In [30]:
from IPython.display import display, HTML
import difflib

def diff_html(old_text, new_text):
    old_words = old_text.split()
    new_words = new_text.split()
    d = difflib.ndiff(old_words, new_words)
    
    output = []
    for i in d:
        if i[0] == ' ':
            output.append(i[2:])
        elif i[0] == '-':
            output.append('<del>' + i[2:] + '</del>')
        elif i[0] == '+':
            output.append('<ins>' + i[2:] + '</ins>')
    return ' '.join(output)

old_text = "The quick brown fox jumps over the lazy dog."
new_text = "The quick brown fox leaps over the lazy dog."

# use IPython's display and HTML functions to display the HTML
display(HTML(diff_html(text1, text2)))


### Strategy 3

In [1]:
resume_work_experience = """
Credit AnalystVine Group, Toronto ON/BNB Financial Group (Merger)                                        Oct 2020 -Present•Manage approval pipeline through daily review and regular follow-up to collect outstanding documentation to ensure mortgages are funding on time•Review documentation for completeness in highest credit quality standards, with compliance to lender policies and procedures•Provide smooth customer service experience in integration with clients and agent team by managing and optimizing interactions•Establish and maintain positive business relationship with external business partners Head Customer Service RepresentativeCIBCMasonville Banking Centre,London,ON                                                    Oct 2018 - Oct 2020•Consistently provided improving customer experience while ensuring error-free transactions in fast pace environment•Assisted CSR team with reconciling and balancing and efficiently correctingerrors•Supported CSR team in balancing system update, proactively created chit-sheet to avoid balancing errors for team•Successfully achieved quality introduction results at 198% over target referrals on SmartReport in 2019•Proactively identified opportunities to introduce client to FSR team to develop financial plansTeaching Assistant     Department of Economics and Finance, UniversityofGuelph                           Sept 2017 - April 2018•Assisted with grading, conducted tutorial sessions and scheduled office hours with students to address course relatedquestions•Worked with minimal supervision while setting priorities and adapting to multiple demands efficiently•Answered students’ questions in a timely manner
"""

In [57]:
system_template = """
You are an experienced career consultalt who helps clients to improve their resumes.
When you are asked to provide evaluation or suggestion, make sure your are critical and specific.
Focus on the use of professional language and the relevancy to the job description.
REMEMBER DO NOT make things up or create fake experiences. 
"""
evaluation_teamplate = """
You are a experienced career consultalt helping clients with the {section} section of their resumes given a job description that the client is applying for.
Give an CRITICAL evaluation of the section focusing on use of professional language and the relevancy to the job description.

{section}: {section_text}
Job Description: {job_description}
"""
suggestion_teamplate = """
You are a experienced career consultalt helping clients with the {section} section of their resumes given a job description that the client is applying for.
You are also given some high level evalutions. Based on the evaluation, provide suggestions on how the client can improve the section. 
Mention the exact wording used and how the client can reword it. 
Try to find all the problems and give specific suggestions.

{section}: {section_text}
Job Description: {job_description}
Evaluate: {evaluation}
"""
revision_teamplate = """
You are a experienced career consultalt helping clients with the {section} section of their resumes given a job description that the client is applying for.
You are also given some suggestions to the resume.Based on the the suggestions, give a revision of the section besed on your suggestions.

{section}: {section_text}
Job Description: {job_description}
Evaluate: {suggestion}
"""
system_prompt = PromptTemplate.from_template(system_template.strip())
evaluation_prompt = PromptTemplate.from_template(evaluation_teamplate.strip())
suggestion_prompt = PromptTemplate.from_template(suggestion_teamplate.strip())
revision_prompt = PromptTemplate.from_template(revision_teamplate.strip())
chain_analyze_3_system = LLMChain(llm=openai, prompt=system_prompt)
chain_analyze_3_evaluation = LLMChain(llm=openai, prompt=evaluation_prompt, output_key="evaluation")
chain_analyze_3_suggestion = LLMChain(llm=openai, prompt=suggestion_prompt, output_key="suggestion")
chain_analyze_3_revision = LLMChain(llm=openai, prompt=revision_prompt, output_key="revised_resume")
chain_analyze_3 = SequentialChain(
    chains=[chain_analyze_3_system, chain_analyze_3_evaluation, chain_analyze_3_suggestion, chain_analyze_3_revision],
    input_variables=["section", "section_text", "job_description"],
    output_variables=["evaluation", "suggestion", "revised_resume"],
    verbose=True
    )

In [59]:
output = chain_analyze_3({"section": "work experience", "section_text": resume_work_experience, "job_description": job_description})

[chain/start] [1:chain:SequentialChain] Entering Chain run with input:
{
  "section": "work experience",
  "section_text": "\nCredit AnalystVine Group, Toronto ON/BNB Financial Group (Merger)                                        Oct 2020 -Present\u2022Manage approval pipeline through daily review and regular follow-up to collect outstanding documentation to ensure mortgages are funding on time\u2022Review documentation for completeness in highest credit quality standards, with compliance to lender policies and procedures\u2022Provide smooth customer service experience in integration with clients and agent team by managing and optimizing interactions\u2022Establish and maintain positive business relationship with external business partners Head Customer Service RepresentativeCIBCMasonville Banking Centre,London,ON                                                    Oct 2018 - Oct 2020\u2022Consistently provided improving customer experience while ensuring error-free transactions in fas

In [62]:
output

{'section': 'work experience',
 'section_text': '\nCredit AnalystVine Group, Toronto ON/BNB Financial Group (Merger)                                        Oct 2020 -Present•Manage approval pipeline through daily review and regular follow-up to collect outstanding documentation to ensure mortgages are funding on time•Review documentation for completeness in highest credit quality standards, with compliance to lender policies and procedures•Provide smooth customer service experience in integration with clients and agent team by managing and optimizing interactions•Establish and maintain positive business relationship with external business partners Head Customer Service RepresentativeCIBCMasonville Banking Centre,London,ON                                                    Oct 2018 - Oct 2020•Consistently provided improving customer experience while ensuring error-free transactions in fast pace environment•Assisted CSR team with reconciling and balancing and efficiently correctingerrors

In [63]:
print(output["revised_resume"])

===================output======================
{'section': 'work experience', 'section_text': '\nCredit AnalystVine Group, Toronto ON/BNB Financial Group (Merger)                                        Oct 2020 -Present•Manage approval pipeline through daily review and regular follow-up to collect outstanding documentation to ensure mortgages are funding on time•Review documentation for completeness in highest credit quality standards, with compliance to lender policies and procedures•Provide smooth customer service experience in integration with clients and agent team by managing and optimizing interactions•Establish and maintain positive business relationship with external business partners Head Customer Service RepresentativeCIBCMasonville Banking Centre,London,ON                                                    Oct 2018 - Oct 2020•Consistently provided improving customer experience while ensuring error-free transactions in fast pace environment•Assisted CSR team with reconciling